#### ALL BLOOD  AND URINE SAMPLE PROCESSING WITHOUT DATE OR TIME

In [108]:
from redcap import Project
import csv
import datetime
import pandas as pd

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'E5FEBF04C14581CF182ABE8107AB8B50'
project = Project(api_url, api_key)

In [109]:
#extract study sample processing data
SampleProcessing = project.export_records(forms=['blood_sample_processing','urine_sample_processing'])

#convert data to Dataframe.
sp_data=pd.DataFrame(SampleProcessing)

In [110]:
# Specify the fields you want to extract base on logics
SP_data=sp_data[['con_participantid_q1','redcap_event_name','bsp_date','bsp_time','usp_date','usp_time','blood_sample_processing_complete','urine_sample_processing_complete']]
SP_data=pd.DataFrame(SP_data)

#st_date as a date data type
SP_data['bsp_date'] = pd.to_datetime(SP_data['bsp_date'], errors='coerce').dt.date
SP_data['bsp_time'] = pd.to_datetime(SP_data['bsp_time'], errors='coerce').dt.time
SP_data['usp_date'] = pd.to_datetime(SP_data['usp_date'], errors='coerce').dt.date
SP_data['usp_time'] = pd.to_datetime(SP_data['usp_time'], errors='coerce').dt.time

condition_arm_1 = ((SP_data['redcap_event_name'] == 'week_20_arm_1') |
                  (SP_data['redcap_event_name'] == 'week_28_arm_1') |
                  (SP_data['redcap_event_name'] == 'week_36_arm_1') |
                  (SP_data['redcap_event_name'] == 'month_1_arm_1')
                  )

condition_arm_2 = (SP_data['redcap_event_name'] == 'month_1_arm_2')

blood_processing_condition = (SP_data['bsp_date'].isnull()) | (SP_data['bsp_time'].isnull()) & (SP_data['blood_sample_processing_complete'] !='0')
urine_processing_condition = (SP_data['usp_date'].isnull()) | (SP_data['usp_time'].isnull()) & (SP_data['urine_sample_processing_complete'] !='0')


df_SP = SP_data[
    (condition_arm_1 & (blood_processing_condition | urine_processing_condition)) |
    (condition_arm_2 & blood_processing_condition  )
]

df_SP = pd.DataFrame(df_SP)

In [111]:
#Save the filtered DataFrame to a CSV file
df_SP.to_csv('SampleProcessing.csv', index=False)